### **Pre-requisite steps:**

1) First, manually download firefox for linux locally somewhere:

http://ftp.mozilla.org/pub/firefox/releases/63.0.3/linux-x86_64/en-US/firefox-63.0.3.tar.bz2

2) then upload as new private "dataset" to kaggle account, making it selectable across kernels

*note: when uploading, make sure to choose "Keep tabular files in original format" *

### 1. installing portable Firefox binary, geckodriver, and selenium library

- 1.1) Manually "+Add data | Your Datasets | firefox-63.0.3.tar.bz2"

note: referencing uploaded binary files as "datasets" automatically places them into "../input" folder

- 1.2) Under Settings section, set Internet = "Internet Conneted"

In [1]:
# WE WILL MAKE NEW SUBFOLDER IN WORKING FOLDER (WHICH ISN'T READ-ONLY)
!mkdir "../working/firefox"
!cp -a "../input/firefox-63.0.3/firefox/." "../working/firefox"
!chmod -R 777 "../working/firefox"
!pip install webdriverdownloader

You are using pip version 18.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from webdriverdownloader import GeckoDriverDownloader

gdd = GeckoDriverDownloader()
gdd.download_and_install("v0.23.0")
!pip install selenium
!apt-get install -y libgtk-3-0 libdbus-glib-1-2 xvfb
!export DISPLAY=:99

3786kb [00:00, 6718.18kb/s]                         


You are using pip version 18.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgtk-3-0 is already the newest version (3.22.11-1).
libgtk-3-0 set to manually installed.
xvfb is already the newest version (2:1.19.2-1+deb9u5).
The following NEW packages will be installed:
  libdbus-glib-1-2
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 206 kB of archives.
After this operation, 343 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian stretch/main amd64 libdbus-glib-1-2 amd64 0.108-2 [206 kB]
Fetched 206 kB in 0s (397 kB/s)          
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package libdbus-glib-1-2:amd64.
(Reading database ... 35433 files and directories currently installed.)
Preparing to unpack .../libdbus-glib-

### 2. Scrape Data from web

In [3]:
# MORE MODULES TO IMPORT
import pandas as pd
import seaborn as sns

from selenium.webdriver.common.by  import By as selenium_By
from selenium.webdriver.support.ui import Select as selenium_Select
from selenium.webdriver.support.ui import WebDriverWait as selenium_WebDriverWait
from selenium.webdriver.support    import expected_conditions as selenium_ec
from selenium.webdriver.firefox.options import Options as selenium_options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities as selenium_DesiredCapabilities
from selenium import webdriver as selenium_webdriver

browser_options = selenium_options()
browser_options.add_argument("--headless")
browser_options.add_argument("--window-size=1920,1080")
browser_options

In [4]:
capabilities_argument = selenium_DesiredCapabilities().FIREFOX
capabilities_argument["marionette"] = True
capabilities_argument

{'browserName': 'firefox', 'marionette': True, 'acceptInsecureCerts': True}

In [5]:
browser = selenium_webdriver.Firefox(
    options=browser_options,
    firefox_binary="../working/firefox/firefox",
    capabilities=capabilities_argument
)
browser

<selenium.webdriver.firefox.webdriver.WebDriver (session="8476522c-75c5-4830-9c67-4bf4735a91ad")>

In [11]:
import time
from datetime import datetime
formatted_data = []

for y in range(2014,2016):
    for m in range(1,13):
        print(y,m)
        # Now we will extract the data from the website via JS return of global variable
        browser.get('https://www.timeanddate.com/weather/canada/toronto/historic?month='+str(m)+'&year='+str(y))
        data = browser.execute_script("return data;")

        # Loop over the results and save results
        for e in data['temp']:
            # Get the date and temp data 
            clean_day = time.strftime('%m',  time.gmtime(e['date']/1000.))
            clean_hour = time.strftime('%H',  time.gmtime(e['date']/1000.))
            formatted_data.append([y,m, int(clean_day), int(clean_hour), e['temp'],(e['temp']-32)*5/9])
            #print([y,m, int(clean_day), int(clean_hour), e['temp'],(e['temp']-32)*5/9])

2014 1
2014 2
2014 3
2014 4
2014 5
2014 6
2014 7
2014 8
2014 9
2014 10
2014 11
2014 12
2015 1
2015 2
2015 3
2015 4
2015 5
2015 6
2015 7
2015 8
2015 9
2015 10
2015 11
2015 12


In [12]:
df = pd.DataFrame(formatted_data,columns=['year','month','day','hour','DegreeF','DegreeC'])
df.to_csv('Temperature_Toronto.csv',index=False)

In [14]:
df.head()

,year,month,day,hour,DegreeF,DegreeC
0,2014,1,1,1,17.204,-8.22
1,2014,1,1,7,15.998,-8.89
2,2014,1,1,13,12.902,-10.61
3,2014,1,1,19,8.798,-12.89
4,2014,1,1,1,7.196,-13.78


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2883 entries, 0 to 2882
Data columns (total 6 columns):
year       2883 non-null int64
month      2883 non-null int64
day        2883 non-null int64
hour       2883 non-null int64
DegreeF    2883 non-null float64
DegreeC    2883 non-null float64
dtypes: float64(2), int64(4)
memory usage: 135.2 KB
